In [1]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from enum import Enum
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser


In [2]:
colunas = ['cd_causa', 'cd_atendimento', 'ds_Acao_Judicial', 'ds_fatos', 'ds_Pedidos', 'ds_Qualificacao']

In [3]:
# LINHA DE SELECAO DO INPUT
df = pd.read_csv(
    "dataset_clinica20252.csv",
    sep="|",               
    encoding="utf-8"     
)

print(df.shape)
print(df.head(20))

(19800, 6)
      cd_causa             cd_atendimento                 ds_Acao_Judicial  \
0   CIB0500064  0825789-84.2025.8.18.0140  90 - ACAO DE REPARACAO DE DANOS   
1   CIB0505587  1004697-72.2025.8.26.0066  90 - ACAO DE REPARACAO DE DANOS   
2   CIB0508201  0800423-07.2025.8.15.0761  90 - ACAO DE REPARACAO DE DANOS   
3   CIB0514647  1004875-69.2025.8.26.0438  90 - ACAO DE REPARACAO DE DANOS   
4   CIB0500604  0010630-50.2025.8.27.2706  90 - ACAO DE REPARACAO DE DANOS   
5   CIB0511068  0092601-36.2025.8.05.0001  90 - ACAO DE REPARACAO DE DANOS   
6   CIB0515030  0801575-51.2025.8.18.0068  90 - ACAO DE REPARACAO DE DANOS   
7   CIB0515018  0801452-38.2025.8.18.0073  90 - ACAO DE REPARACAO DE DANOS   
8   CIB0515702  0800895-81.2025.8.10.0038  90 - ACAO DE REPARACAO DE DANOS   
9   CIB0514229  0148716-17.2025.8.04.1000  90 - ACAO DE REPARACAO DE DANOS   
10  CIB0514230  0148734-38.2025.8.04.1000  90 - ACAO DE REPARACAO DE DANOS   
11  CIB0524760  0161843-22.2025.8.04.1000  90 - ACAO 

In [4]:
from utils.vector_store import search_vector_store

def exemplos_contexto(query: str):
    positivos, negativos = search_vector_store(query)
    return positivos, negativos

/home/danilo/.local/lib/python3.10/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [5]:
exemplos_positivos, exemplos_negativos = exemplos_contexto(df['ds_fatos'].iloc[0])
print(exemplos_positivos, exemplos_negativos)

[{'texto': '\n        A parte autora, pessoa sem leitura, é titular de um benefício previdenciário que é a fonte do sustento seu e de sua família. \n        Com o fim de aclarar a quantidade e natureza dos descontos que vem experimentando em seu benefício, procurou ajuda profissional \n        para que fossem feitos os requerimentos administrativos e demais investigações por um profissional especializado e de sua confiança.  \n        Nesta oportunidade, e em observância ao extrato fornecido pela Previdência Social (doc. em anexo), foi constatado que seu benefício \n        sofreu e/ou vem sofrendo descontos por parte da requerida em decorrência de empréstimo consignado, com os seguintes contratos: \n        NOME: LUZIA BATISTA DA SILVA BANCO REQUERIDO: BANCO BRADESCO S.A. ? 1º Contrato n°386039739, no valor mensal fixo de R$150,77, \n        com vigência de 11/02/2020 - 30/01/2024, com o total de 48 parcelas pagas até a presente data, totalizando o valor de R$7236,96\n        ', 'scor

In [7]:
class BinaryChoice(str, Enum):
    SIM = "SIM"
    NAO = "NAO"

class Classificador(BaseModel):
    binary_score: BinaryChoice = Field(
        description="SIM para crédito consignado como causa raiz, NAO para documentos com outra causa principal"
    )


In [ ]:
def classificar_linhas(texto: str) -> str:
    model = ChatOpenAI(model="gpt-4o", temperature=0)

    pos, neg = exemplos_contexto(texto)
    ctx_true = "\n".join([f"{ex['texto']} - score: {ex['score']}" for i, ex in enumerate(pos)])
    ctx_false = "\n".join([f"{ex['texto']} - score: {ex['score']}" for i, ex in enumerate(neg)])

    parser = PydanticOutputParser(pydantic_object=Classificador)
    
    prompt_template = '''
    Você é um especialista em análise de documentos jurídicos brasileiros.

    Analise o texto abaixo e classifique se o caso trata de CRÉDITO CONSIGNADO.

    CRÉDITO CONSIGNADO é caracterizado por:
    - Contratação VOLUNTÁRIA de empréstimo com desconto em folha de pagamento
    - Autor SOLICITOU o empréstimo e AUTORIZOU os descontos
    - Discussão sobre TAXAS DE JUROS, PRAZOS ou CONDIÇÕES do empréstimo consignado
    - Margem consignável excedida
    - Problemas com portabilidade de crédito consignado

    NÃO é crédito consignado:
    - Descontos NÃO AUTORIZADOS de seguros, cartões ou serviços bancários
    - Cobranças indevidas de produtos não contratados
    - Venda casada
    - Negativação indevida
    - Cartão de crédito comum (não consignado)

    Use como contexto os exemplos abaixo:
    Exemplos POSITIVOS (CRÉDITO CONSIGNADO):
    {ctx_true}

    Exemplos NEGATIVOS (NÃO CRÉDITO CONSIGNADO):
    {ctx_false}

    Utilize o score dos exemplos para ajudar na decisão.

    Texto para analisar: {texto}

    {format_instructions}
    '''
        
    prompt = ChatPromptTemplate.from_template(
        template=prompt_template,
        partial_variables={
            "format_instructions": parser.get_format_instructions(),
            "ctx_true": ctx_true,
            "ctx_false": ctx_false}
    )
    
    

    chain = prompt | model | parser
    #rendered = prompt.format(texto=texto)
    #print("=== PROMPT RENDERIZADO ===")
    #print(rendered)
    #print("===========================")
    
    resultado = chain.invoke({"texto": texto})
    return resultado.binary_score.value

In [33]:
texto = df['ds_fatos'].iloc[0]

classificar_linhas(texto)

=== PROMPT RENDERIZADO ===
Human: 
    Você é um especialista em análise de documentos jurídicos brasileiros.

    Analise o texto abaixo e classifique se o caso trata de CRÉDITO CONSIGNADO.

    CRÉDITO CONSIGNADO é caracterizado por:
    - Contratação VOLUNTÁRIA de empréstimo com desconto em folha de pagamento
    - Autor SOLICITOU o empréstimo e AUTORIZOU os descontos
    - Discussão sobre TAXAS DE JUROS, PRAZOS ou CONDIÇÕES do empréstimo consignado
    - Margem consignável excedida
    - Problemas com portabilidade de crédito consignado

    NÃO é crédito consignado:
    - Descontos NÃO AUTORIZADOS de seguros, cartões ou serviços bancários
    - Cobranças indevidas de produtos não contratados
    - Venda casada
    - Negativação indevida
    - Cartão de crédito comum (não consignado)

    Use como contexto os exemplos abaixo:
    Exemplos POSITIVOS (CRÉDITO CONSIGNADO):
    
        A parte autora, pessoa sem leitura, é titular de um benefício previdenciário que é a fonte do susten

'NAO'

In [27]:
df_validado_true = df.loc[
    df['ds_fatos'].str.contains("A parte Autora é beneficiária da Previdência Social, sendo tal sua única fonte de renda.  Portanto, por", 
    na=False)]

texto = df_validado_true['ds_fatos'].iloc[0]

classificar_linhas(texto)

'SIM'